In [1]:
import pandas as pd

## 레진 데이터 챌린지 2017

오늘의 주제는 [레진 데이터 챌린지 2017](http://tech.lezhin.com/events/data-challenge-pyconkr-2017)의 데이터를 바탕으로, 레진코믹스의 구매량과 주 사용 플랫폼과 작품 등의 상관관계를 분석하는 것입니다. 총 데이터는 650,965 건이며, 이 데이터를 판다스로 활용해 분석할 수 있어야 합니다.

[원래 경진대회](http://tech.lezhin.com/events/data-challenge-pyconkr-2017) 에서는 머신러닝을 활용해 구매 예측 여부를 판단하였지만, 이번 과제에서는 머신러닝을 사용하지 않고, 판다스만을 사용해 분석할 것입니다.

데이터는 [다음의 링크](https://storage.googleapis.com/lz-insight/pycon17/dataset/lezhin_dataset_v2_training.tsv.gz)에서 다운받으실 수 있습니다.

** 목표 **

주어진 데이터를 활용하여, 타 팀에서 요청한 10여개의 질문에 대한 답변을 줘야 합니다. 툴은 파이썬과 판다스를 사용하며, 타 팀에서 직관적으로 이해할 수 있도록 가능한 깔끔하고 정리된 결과가 나오면 좋습니다.

** 데이터셋 **
* 파일 포맷: TSV
* 파일 용량: 228M (압축해서 26M)
* 샘플 수: 650,965 건
* feature 수: 167 개
  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 6 : 목록 진입시점 방문 총 세션 수 (범위별로 부여된 순차 ID)
  * 7 : 작품을 나타내는 해쉬
  * 8-10 : 개인정보
  * 11-110 : 주요 작품 구매 여부
  * 111 : 작품 태그 정보
  * 112 : 구매할 때 필요한 코인
  * 113 : 완결 여부
  * 114-123 : 스케쥴 정보
  * 124-141 : 장르 정보
  * 142 : 해당 작품의 마지막 에피소드 발행 시점 (범위별로 부여된 순차 ID)
  * 143 : 단행본 여부
  * 144 : 작품 발행 시점 (범위별로 부여된 순차 ID)
  * 145 : 총 발행 에피소드 수 (범위별로 부여된 순차 ID)
  * 146-151 : 작품 태그 정보
  * 152-167 : 유저의 성향 정보 (과거에 구매를 했을 때만 기록)

## 데이터 로딩하기

In [2]:
data = pd.read_csv("lezhin_dataset_v2_training.tsv", sep="\t", header=None, nrows=100000)

print(data.shape)
data.head()

(100000, 167)


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,NaN,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,NaN,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,NaN,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN


### 1. 사용 플랫폼 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D

** 1. 전체 데이터(650,965 건)에서 구매한 사용자 / 구매하지 않은 사용자의 인원 수를 찾아주세요. **

In [3]:
str = ["구매여부", "A", "B", "C", "D"]
for i in range(6, data.shape[1]+1) :
    str.append(i)
    
data.columns = str

print(data.shape)
data.head()

(100000, 167)


,구매여부,A,B,C,D,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,167
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,NaN,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,NaN,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,NaN,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN


In [4]:
data["구매여부"].value_counts()

0    100000
Name: 구매여부, dtype: int64

** 2. 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수를 찾아주세요. . **

즉, 표의 세로축에는 구매 여부(0, 1)가, 가로축에서는 각 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수가 나왔으면 좋겠습니다.

In [5]:
group_category = data.groupby('구매여부')

In [6]:
group_category["A", "B", "C", "D"].sum()

,A,B,C,D
구매여부,,,,
0,0,0,28477,71523


** 3. 2번 데이터에서 총 인원수는 물론, 평균 구매율도 함께 계산해주세요.**

힌트: 총 인원 수를 sum이라고 한다면, 평균 구매율은 mean이라고 할 수 있습니다.



In [7]:
group_category["A", "B", "C", "D"].mean()

,A,B,C,D
구매여부,,,,
0,0.0,0.0,0.28477,0.71523


** 4. '사용 플랫폼' 이라는 새로운 컬럼을 만들어주세요 **

데이터를 분석한 결과, 두 개 이상의 플랫폼에서 동시에 방문하는 경우는 찾을 수 없었습니다.

In [8]:
# 동시에 두 개 이상의 플랫폼에서 방문 + 구매하는 경우(> 1)는 없다는 것을 확인할 수 있다.

# 1 == 사용 플랫폼 A
# 2 == 사용 플랫폼 B
# 3 == 사용 플랫폼 C
# 4 == 사용 플랫폼 D
((data["A"] + data["B"] + data["C"] + data["D"]) > 1).sum()

0

그러므로 사용 플랫폼 A, 사용 플랫폼 B, 사용 플랫폼 C, 사용 플랫폼 D를 따로 나누지 않고, **사용 플랫폼**이라는 하나의 컬럼으로 합쳤으면 좋겠습니다. 해당 컬럼에는 A, B, C, D 값 중 하나가 들어갑니다.


In [9]:
data.loc[data["A"] == 1, "사용 플랫폼"] = "A"
data.loc[data["B"] == 1, "사용 플랫폼"] = "B"
data.loc[data["C"] == 1, "사용 플랫폼"] = "C"
data.loc[data["D"] == 1, "사용 플랫폼"] = "D"
data.head()

,구매여부,A,B,C,D,6,7,8,9,10,...,159,160,161,162,163,164,165,166,167,사용 플랫폼
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN,D
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN,D
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN,D
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN,D
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN,D


### 2. 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 112 : 구매할 때 필요한 코인 (이하 구매 코인)
 

여기서 코인 1개의 가격은 100원이라고 가정합니다.

** 5. 총 작품의 갯수를 구해주세요. **

In [10]:
data[7].describe()

count       100000
unique        2382
top       8415CB0B
freq          1170
Name: 7, dtype: object

** 6. 가장 많이 구매한 작품 Top 10을 찾아주세요. **

In [11]:
data[7].value_counts().head(10)

8415CB0B    1170
85803B60     914
D619AC7D     873
5977BD95     848
0365FD34     845
0757A410     823
707F7107     791
37C9A863     781
6983846D     777
0F130215     758
Name: 7, dtype: int64

** 7. 정 반대로, 가장 적게 구매한 작품 Top10을 구해주세요. **

In [12]:
data[7].value_counts().tail(10)

596719D1    1
60AF70B4    1
5F964518    1
FE3E04FB    1
F64768BC    1
E0CF1F47    1
E5F6AD6C    1
22239F3B    1
CA028CA8    1
DE9E7FC1    1
Name: 7, dtype: int64

** 8. 구매 코인과 코인 가격을 고려하여, 가장 높은 매출을 달성한 작품 Top 10을 구해주세요. **

한 번 구매했을 때 가격은 구매 코인 * 가격(100 원)이라고 볼 수 있습니다. 가령 구매 코인이 4라면 가격은 400 원이 됩니다.

In [17]:
# data["매출"] = data[112]*100
data_coin = data.sort_values(by=7, ascending=False)
data_coin

,구매여부,A,B,C,D,6,7,8,9,10,...,160,161,162,163,164,165,166,167,사용 플랫폼,매출
3145,0,0,0,0,1,10,FFEABD22,2D37CF89,0,NaN,...,NaN,NaN,NaN,NaN,0.0448,0.0057,NaN,NaN,D,0
42430,0,0,0,0,1,36,FFEABD22,C7390752,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,0
13357,0,0,0,0,1,139,FFEABD22,39FF8AC1,1,NaN,...,NaN,NaN,NaN,NaN,0.0558,0.0065,NaN,NaN,D,0
9057,0,0,0,0,1,12,FFEABD22,E0BCD0FF,1,NaN,...,NaN,NaN,NaN,NaN,0.0603,NaN,NaN,NaN,D,0
39163,0,0,0,0,1,319,FFEABD22,EEF21E68,1,NaN,...,NaN,NaN,NaN,NaN,0.0194,0.0046,NaN,NaN,D,0
75014,0,0,0,1,0,12,FFD52F3C,A189CC17,1,NaN,...,NaN,NaN,NaN,NaN,0.3396,0.1145,NaN,NaN,C,0
48187,0,0,0,0,1,44,FFD52F3C,58E1E504,0,NaN,...,0.0827,NaN,NaN,NaN,0.0354,0.0142,NaN,NaN,D,0
80442,0,0,0,1,0,171,FFD52F3C,0033C556,1,8884750A,...,NaN,NaN,NaN,NaN,0.0247,NaN,NaN,NaN,C,0
61359,0,0,0,0,1,70,FFC5BE2E,C10FD24C,1,NaN,...,0.0456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,300
18284,0,0,0,0,1,16,FFC5BE2E,8B61ADFD,1,NaN,...,NaN,NaN,NaN,NaN,0.0132,NaN,NaN,NaN,D,300


** 9. 몇몇 주요 작품들의 사용 플랫폼별 구매 횟수를 구해주세요 **

다음의 다섯 개 아이디(**D91D1B4D**, **72B32A1F**, **06138BC5**, **7F7DDD7D**, **DBB99DB2**)를 주요 작품으로 간주합니다.

해당 다섯 개의 작품이 사용 플랫폼 A~D에서 구매한 횟수를 각각 구해주세요.

In [ ]:
tab[["D91D1B4D", "72B32A1F", "06138BC5", "7F7DDD7D", "DBB99DB2"]]

** 10. 100회 이상 구매한 작품 중, 가장 구매 확률이 높은 작품 top 10을 찾아주세요. **

팁: 세 개를 동시에 시도하지 않고, 1) 100회 이상 구매하지 않은 작품을 제거한 뒤, 2) 1번에서 작품당 구매확률을 구하고, 3) 2번에서 구매 확률로 정렬하는 방식으로 일을 쪼개서 작업하면 조금 더 쉽게 구현할 수 있습니다.

In [ ]:
# 100회 이상 구매한 작품만 필터링
top_comics = comics["작품 번호"].value_counts()
top_comics = top_comics[top_comics >= 100]
top_comics = top_comics.index

print(len(top_comics))
top_comics

In [ ]:
top_comics = comics[comics["작품 번호"].isin(top_comics)]

print(top_comics.shape)
top_comics.head()

In [ ]:
# 작품당 구매 확률을 계산
tab = pd.crosstab(top_comics["구매 여부"], top_comics["작품 번호"])
tab = tab.apply(lambda r: r / r.sum(), axis=0)
tab

In [ ]:
# 구매 확률이 높은 쪽으로 정렬
top = tab.loc[1].sort_values(ascending=False)
top10 = top.head(10)
top10

### 3. 주요 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 11-110 : 주요 작품 구매 여부. (이하 주요 작품 1 ~ 주요 작품 100)

In [ ]:
import numpy as np

related_comices = data.loc[:, 10:109].astype(np.int32)
related_comices.columns = [f"주요 작품 {i+1}" for i in range(related_comices.shape[1])]

print(related_comices.shape)
related_comices.head()

** 11. 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [ ]:
related_comices.sum(axis=0).sort_values(ascending=False).head(10)

** 12. 구매 여부를 기준으로, 구매하지 않은 사용자는 배제하고 구매한 사용자 기준으로 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [ ]:
related_comices["구매 여부"] = data[0]

purchased_related_comices = related_comices[related_comices["구매 여부"] == 1]

print(purchased_related_comices.shape)
purchased_related_comices.head()

In [ ]:
purchased_related_comices.sum(axis=0).sort_values(ascending=False).head(10)

** 13. 구매 횟수와 상관 없이, 구매 확률이 가장 높은 주요 작품 Top 10을 찾아주세요. **

가령 구매를 20번 하건 30번 하건 전부 1번으로 가정 합니다. 또 다른 예를 들자면, 전체 650,965 개의 데이터에서 주요 작품 1번을 6,509번 구매했다면 구매 확률은 0.01(1%)로 가정합니다.

In [ ]:
related_comices_binary = related_comices.astype('bool').astype('int')
related_comices_binary.mean(axis=0).sort_values(ascending=False).head(10)

** 14. 작품 번호별로, 구매 횟수가 가장 높은 주요 작품(주요 작품 1 ~ 100)의 Top 10을 찾아주세요. **

또한 작품 번호는 누적 구매 수가 가장 많은 순으로 정렬, 주요 작품도 누적 구매 수가 가장 많은 순으로 정렬합니다.

In [ ]:
related_comices["작품 번호"] = data[6]

table = pd.pivot_table(related_comices,
                       index="작품 번호",
                       values=[f"주요 작품 {i+1}" for i in range(100)],
                       aggfunc=np.sum)

table.head(10)

In [ ]:
index = table.sum(axis=1).sort_values(ascending=False).index
columns = table.sum(axis=0).sort_values(ascending=False).index

table.loc[index, columns].head(10)

### ToDo

* categorical 사용해보기